In [1]:
using PyCall
using CSV
using Dates
using Distributions
using DataFrames
using .Threads
using CairoMakie
using StatsBase

In [2]:
include("C:/Users/hamid/OneDrive/Coding/RiskLabAI/RiskLab Code/2018-03Labels/labeling.jl")

SystemError: SystemError: opening file "C:\\Users\\hamid\\OneDrive\\Coding\\RiskLabAI\\RiskLab Code\\2018-03Labels\\labeling.jl": No such file or directory

In [3]:
include("C:\\Users\\hamid\\OneDrive\\Coding\\RiskLabAI\\RiskLab Code\\2018-03Labels\\labeling.jl")
include("C:\\Users\\hamid\\OneDrive\\Coding\\RiskLabAI\\RiskLab Code\\2018-20MultiprocessingVectorization\\HPC.jl")
include("SampleWeight.jl")

SystemError: SystemError: opening file "C:\\Users\\hamid\\OneDrive\\Coding\\RiskLabAI\\RiskLab Code\\2018-03Labels\\labeling.jl": No such file or directory

In [12]:
py"""
import yfinance
import datetime as dt
def get_data():
  enddate = dt.datetime.strptime("2021-01-01", "%Y-%m-%d").date()
  startdate = enddate - dt.timedelta(days=360*2) # for 5 years

  data = yfinance.download(tickers="AAPL", period="700d", interval="1h")
  return data
"""
df = py"get_data"();

py"""
import pandas as pd
import numpy as np

def get_ohlcv(df_group):
    ohlc = df_group['Close'].ohlc()
    ohlc['volume'] = df_group['Volume'].sum()
    return ohlc

def get_time_bar(df, freq="5Min"):
    df_group = df.groupby(pd.Grouper(freq=freq))
    
    ohlcv = get_ohlcv(df_group).dropna()
    return ohlcv.index , ohlcv.to_numpy()
"""

py"""
def reset_data_time(data):
  df  = data.copy()
  df.set_index(df.index.strftime("%Y-%m-%d"))
  return df
"""

indexs ,ohlcv = py"get_time_bar"(df, freq="1B")
index = indexs.strftime("%Y-%m-%d");
data = DataFrame(Dates = Date.(index,dateformat"y-m-d"),open = ohlcv[:,1] , high = ohlcv[:,2] , low = ohlcv[:,3] , close = ohlcv[:,4] , volume = ohlcv[:,5]);
close = select(data,["Dates" , "close"]);

[*********************100%***********************]  1 of 1 completed

In [13]:
numDays = 21
rets,stds = dailyvol(close,32);
tEvents = events(close, 2);
t1 = verticalbarrier(close,tEvents,numDays);
events_ = events(close, tEvents, 1, stds, 0.0; timestamp = t1.x2);

UndefVarError: UndefVarError: dailyvol not defined

In [ ]:
timestamp = select(events_,["date" , "timestamp"]);

In [ ]:
curr = concurrencyEvents(close.Dates , timestamp ,timestamp.date);

In [ ]:
sample_weight = sampleWeight(timestamp,curr,timestamp.date);

In [ ]:
rets.x1 = Date.(rets.x1)
rets = filter(row -> row[:x1] in curr.date,rets)
returns = DataFrame(date = rets.x1 , returns = rets.x2)
sample_weight_ret = sampleWeight(timestamp,curr,returns,timestamp.date)

In [ ]:
timeDecay(sample_weight_ret ;clfLastW = -0.5)

In [ ]:
function RandomTimestamp(nObservation , nBars , maximumHolding)
    timestamp = DataFrame(date = zeros(nObservation) , timestamp = zeros(nObservation))
    for i in 1:nObservation
        date = rand(1:nBars)
        t = date + rand(1:maximumHolding)
        timestamp.date[i] = date
        timestamp.timestamp[i] = t
    end
    sort!(timestamp,[:date])
    return timestamp
end
function MonteCarloSimulationforSequentionalBootstraps(nObservation , nBars , maximumHolding)
    timestamp = RandomTimestamp(nObservation , nBars , maximumHolding)
    barIndex  = [i for i in 1:maximum(timestamp.timestamp) + 1]
    indexMatrix_ = indexMatrix(barIndex,timestamp)
    ϕ = sample(1:size(indexMatrix_)[2] ,size(indexMatrix_)[2])
    standardUniqueness = mean(averageUniqueness(indexMatrix_[:,ϕ]))
    ϕ = sequentialBootstrap(indexMatrix_,size(indexMatrix_)[2])
    sequentioalUniqueness = mean(averageUniqueness(indexMatrix_[:,ϕ]))
    return standardUniqueness ,sequentioalUniqueness
end

In [ ]:
function SimulateSequentionalVsStandardBootstrap(iteration,nObservation , nBars , maximumHolding)
    seqU = [[] for i in 1:nthreads()]
    stdU = [[] for i in 1:nthreads()]
    @threads for i in 1:iteration
        standardUniqueness ,sequentioalUniqueness = MonteCarloSimulationforSequentionalBootstraps(nObservation , nBars , maximumHolding)
        append!(seqU[threadid()],sequentioalUniqueness)
        append!(stdU[threadid()],standardUniqueness)
    end
    seqU = vcat(seqU...)
    stdU = vcat(stdU...)
    return seqU,stdU
end
seqU , stdU = SimulateSequentionalVsStandardBootstrap(1000000,10 , 100 , 5);

In [ ]:
f = Figure()
Axis(f[1, 1])

density!(Float64.(seqU), color = (:red, 0.3),
    strokecolor = :red, strokewidth = 3, strokearound = true)
density!(Float64.(stdU), color = (:blue, 0.3),
    strokecolor = :blue, strokewidth = 3, strokearound = true)
f